In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Reading in the training and testing data set
training_data_df = pd.read_csv('C:/Users/LianJ/Desktop/Tensorflow/Exercise Files/03/sales_data_training.csv', dtype = float)
test_data_df = pd.read_csv('C:/Users/LianJ/Desktop/Tensorflow/Exercise Files/03/sales_data_test.csv', dtype = float)

In [3]:
#Separate the training and testing data set
X_training = training_data_df.drop('total_earnings', axis = 1).values
Y_training = training_data_df[['total_earnings']].values
X_testing = test_data_df.drop('total_earnings', axis = 1).values
Y_testing = test_data_df[['total_earnings']].values

In [4]:
#Establish the scaler
X_scaler = MinMaxScaler(feature_range = (0, 1))
Y_scaler = MinMaxScaler(feature_range = (0, 1))

In [5]:
#Scale the data
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

In [6]:
X_scaled_testing = X_scaler.fit_transform(X_testing)
Y_scaled_testing = Y_scaler.fit_transform(Y_testing)

In [24]:
#Setting up hyperparameters
RUN_NAME = "run 3 with 20 nodes"
learning_rate = 0.001
training_epochs = 100
display_step = 5

In [8]:
number_of_inputs = 9
number_of_outputs = 1

In [25]:
layer_1_nodes = 30
layer_2_nodes = 100
layer_3_nodes = 50

In [10]:
#Input layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape = (None, number_of_inputs)) #None: mix batch of any size

In [11]:
#layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable(name="weights1", shape = [number_of_inputs, layer_1_nodes], 
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape = [layer_1_nodes], initializer = tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
#layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable(name="weights2", shape = [layer_1_nodes, layer_2_nodes], 
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape = [layer_2_nodes], initializer = tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

In [13]:
#layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable(name="weights3", shape = [layer_2_nodes, layer_3_nodes], 
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape = [layer_3_nodes], initializer = tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

In [14]:
#Output layer
with tf.variable_scope('output'):
    weights = tf.get_variable(name="weights4", shape = [layer_3_nodes, number_of_outputs], 
                              initializer = tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape = [number_of_outputs], initializer = tf.zeros_initializer())
    prediction = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)

In [15]:
#Cost function
with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

In [16]:
#Optimizer
with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [26]:
#Logging our data for tensorboard
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    tf.summary.histogram('predicted_value', prediction)
    summary = tf.summary.merge_all()

In [27]:
#Run our sesssion
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    
    training_writer = tf.summary.FileWriter('C:/Users/LianJ/Documents/event_logs/{}/training'.format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter('C:/Users/LianJ/Documents/event_logs/{}/testing'.format(RUN_NAME), session.graph)
    
    for epoch in range(training_epochs):
        session.run(optimizer, feed_dict = {X: X_scaled_training, Y: Y_scaled_training})
        
        if epoch%10 == 0:
            training_cost, training_summary = session.run([cost, summary], feed_dict = {X: X_scaled_training, Y: Y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict = {X: X_scaled_testing, Y: Y_scaled_testing})
            
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            
            print(epoch, training_cost, testing_cost)
            
    print("Training is complete")

0 0.04354622 0.066165976
10 0.0111678755 0.016873544
20 0.0046849395 0.007902198
30 0.0032234793 0.00596624
40 0.0014869118 0.0030735272
50 0.00094026444 0.0023975149
60 0.00056467665 0.0017647168
70 0.00036395268 0.0017189765
80 0.00025464373 0.001592644
90 0.00018591745 0.0014057918
Training is complete
